In [ ]:

#Avg_SentenceLength_Mid_agent
def plot_variables(samples, idx_1, idx_2, target_column):
    plt.figure(figsize=(20,10))
    plt.scatter(samples.iloc[:, idx_1], samples.iloc[:, idx_2], c=samples[target_column], s=300)
    for idx in np.arange(len(samples)):
        plt.annotate(str(idx), (samples.iloc[idx, idx_1], samples.iloc[idx, idx_2]))
    plt.xlabel(samples.columns[idx_1])
    plt.ylabel(samples.columns[idx_2])
    plt.colorbar()
    plt.show()

col1 = 'Avg_AnswersDelay_Mid'
col2 = 'Avg_AnswersDelay_Mid_agent'    

plot_variables(all_np_df, list(all_np_df.columns).index(col1), list(all_np_df.columns).index(col2), 'Presence Score')

In [ ]:
# from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import ShuffleSplit
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.dummy import DummyClassifier

if 'PresenceClass_5_kmeans' in all_p_df.columns:
    all_np_df = all_np_df.drop(['PresenceClass_5_kmeans'], axis=1)
    
dataset = all_p_df
featureset = get_features_set('Doctor+Agent', '157015-A')
print(featureset)
nb = 100
ds_splits = prepare_train_data(dataset, featureset,
                                          'presence', bins=2, strategy='uniform', upsample=None, test_set=0.1, n_splits=nb,  verbose=0)
#print('Generated %d splits' % len(ds_splits))
scores = np.zeros((nb, 7))

for idx, (X, y, X_test, y_test) in enumerate(ds_splits):
    #print(np.unique(y, return_counts=True))
    print('X ' + str(X.shape))
    sc = StandardScaler()
    X = sc.fit_transform(X)
    X_test = sc.transform(X_test)
    print("y : %s" % str(np.unique(y, return_counts=True)))
    
    from sklearn.svm import LinearSVC
    from sklearn.feature_selection import SelectFromModel

    lsvc = LinearSVC(C=0.1, penalty="l1", dual=False).fit(X, y)
    model = SelectFromModel(lsvc, prefit=True)
    X_new = model.transform(X)
    print('LSVC feature selection from %s to %s' % (str(X.shape), str(X_new.shape)))
    
    from sklearn.model_selection import StratifiedKFold
    from sklearn.feature_selection import RFECV
    
    lsvc = LinearSVC(C=0.1, penalty="l1", dual=False)
    rfecv = RFECV(estimator=lsvc, step=1, cv=StratifiedKFold(10), scoring='f1')
    rfecv.fit(X, y)

    print("Optimal number of features : %d" % rfecv.n_features_)

    # Plot number of features VS. cross-validation scores
    plt.figure()
    plt.xlabel("Number of features selected")
    plt.ylabel("Cross validation score (nb of correct classifications)")
    plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
    plt.show()
    
    lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto', n_components=5)
    lda.fit(X, y)
    y_pred = lda.predict(X_test)
    scores[idx, 0] = balanced_accuracy_score(y_test, y_pred, adjusted=False)
    scores[idx, 1] = balanced_accuracy_score(y_test, y_pred, adjusted=True)
    print('X ' + str(X.shape) + ', coef_ ' + str(lda.coef_))
    
    qda = QuadraticDiscriminantAnalysis()
    qda.fit(X, y)
    y_pred = qda.predict(X_test)
    scores[idx, 2] = balanced_accuracy_score(y_test, y_pred, adjusted=False)
    scores[idx, 3] = balanced_accuracy_score(y_test, y_pred, adjusted=True)
    print(str(scores[idx]))
    #trans = lda.transform(X)
    #trans_test = lda.transform(X_test)
    #plt.scatter(trans[:, 0], trans[:, 1], c=y, marker='o')
    #plt.scatter(trans_test[:, 0], trans_test[:, 1], c=y_test, marker='x')
    #plt.show()
    
    dummy = DummyClassifier(strategy='uniform', random_state=RANDOM_STATE)
    dummy.fit(X,y)
    y_pred = dummy.predict(X_test)
    scores[idx, 4] = balanced_accuracy_score(y_test, y_pred, adjusted=False)
    
    dummy = DummyClassifier(strategy='stratified', random_state=RANDOM_STATE)
    dummy.fit(X,y)
    y_pred = dummy.predict(X_test)
    scores[idx, 5] = balanced_accuracy_score(y_test, y_pred, adjusted=False)
    
    dummy = DummyClassifier(strategy='most_frequent', random_state=RANDOM_STATE)
    dummy.fit(X,y)
    y_pred = dummy.predict(X_test)
    scores[idx, 6] = balanced_accuracy_score(y_test, y_pred, adjusted=False)    
means = np.mean(scores, axis=0)
print('mean ' + str(means))
best_rnd = np.max(means[4:-1])
print('best random' + str(best_rnd))
print('adjusted with best rnd ' + str(means[0] - best_rnd))

print('std ' + str(np.std(scores, axis=0) / nb))

In [ ]:
try:
    del sys.modules['feutils']
    del sys.modules['config']
except:
    pass
from feutils import DataHandler
from feutils import JNCC2Wrapper
import config

if 'PresenceClass_5_kmeans' in all_p_df.columns:
    all_np_df = all_np_df.drop(['PresenceClass_5_kmeans'], axis=1)
    
dataset = all_np_df
featureset = get_features_set('Doctor', 'Multimodal-A')
dh = DataHandler('__tests', 'jncc2', 0)
print(featureset)
nb = 100
jncc = JNCC2Wrapper(dh)
ds_splits = prepare_train_data(dataset, featureset,
                                          'presence', bins=5, strategy='kmeans', upsample='imb_smotenc', test_set=0.1, n_splits=nb, clf=jncc, verbose=0)
#print('Generated %d splits' % len(ds_splits))
scores = np.zeros((nb, 2))

for idx, (X, y, X_test, y_test) in enumerate(ds_splits):
    #print(np.unique(y, return_counts=True))
    #print('X ' + str(X.shape))
    #sc = StandardScaler()
    #X = sc.fit_transform(X)
    #X_test = sc.transform(X_test)
    
    #print(dataset['Presence Score'])
    #print(dataset['Presence Score'].iloc[y.index])

    jncc_cv, y_trues_cv = jncc.cv('train.arff', 0)
    
    jncc_result, y_true_predict = jncc.predict('train.arff', 'test.arff', 0)
    
    # make it regression
    y_train, enc = discretize(dataset['Presence Score'], 3, 'kmeans')
    idx_train = y.index
    idx_test = y_test.index
    y = y_train[idx_train]
    y_test = y_train[idx_test]
    y_reg = dataset['Presence Score'].iloc[idx_train]
    y_reg_test = dataset['Presence Score'].iloc[idx_test]
    
    from sklearn import linear_model
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.feature_selection import RFE
    
    #reg = linear_model.Lasso(alpha=0.1)
    reg = RandomForestRegressor(n_estimators=500)
    rfe = RFE(estimator=reg, n_features_to_select=2, step=1)
    
    #reg = LogisticRegression()
    rfe.fit(X, y_reg)
    print(zip(featureset.to_numpy(), rfe.support_))
    print("Regression score %f" % reg.score(X_test, y_reg_test))
    y_reg_pred = reg.predict(X_test)
    print(zip(featureset.to_numpy(), reg.feature_importances_))
    
    #print(enc.inverse_transform(y_train))
    #print("shape " + str(y_train.shape))
    #print("shape pred " + str(np.array(y_reg_pred).reshape(-1, 1).shape))
    y_pred = enc.transform(np.array(y_reg_pred).reshape(-1, 1))
    
    scores[idx, 0] = balanced_accuracy_score(y_test, y_pred, adjusted=False)
    scores[idx, 1] = balanced_accuracy_score(y_test, y_pred, adjusted=True)
    
print('mean ' + str(np.mean(scores, axis=0)))
print('std ' + str(np.std(scores, axis=0) / nb))

In [ ]:
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.preprocessing import LabelBinarizer

def compute_lrap(y_true, y_pred, labels):
    if type(y_true) is np.ndarray:
        y_true = pd.DataFrame(y_true)
    if type(y_pred) is np.ndarray:
        y_pred = pd.DataFrame(y_pred)
    lb = LabelBinarizer()
    lb.fit(labels)
    y_score = np.zeros(y_pred.values.shape)
    for i in np.arange(y_score.shape[1]):
        col = y_pred.iloc[:, i]
        binarized = lb.transform(col)
        y_score = y_score + (binarized.astype(float) / (i+1))
    y_true_bin = lb.transform(y_true.values.reshape(-1,1))
    return label_ranking_average_precision_score(y_true_bin, y_score)

print(compute_lrap(y_test, jncc_result, [0,1,2,3,4]))
cv_lraps = [compute_lrap(y_trues_cv[i], jncc_cv[i], [0,1,2,3,4]) for i in np.arange(len(jncc_cv))]
cv_lraps = np.array(cv_lraps)
print(cv_lraps)
print(cv_lraps.mean())
print(cv_lraps.std())
print(balanced_accuracy_score(y_true_predict, jncc_result.iloc(axis=1)[0], adjusted=True))

In [ ]:
# todo "Avg_IPUlen_Begin", "Avg_IPUlen_Middle", "Avg_IPUlen_End" removed because of wrong values
#__features = ["Expert", "Head_Entropy_Start", "Head_Entropy_Mid", "Head_Entropy_End", "Avg_HandEntropy_Begin",
#             "Avg_HandEntropy_Mid", "Avg_HandEntropy_End", "Avg_SentenceLength_Begin", "Avg_SentenceLength_Mid",
#             "Avg_SentenceLength_End",  "Ratio1_Begin", "Ratio1_Mid", "Ratio1_End", "Ratio2_Begin", "Ratio2_Mid", 
#                    "Ratio2_End", "Duration"]
"""mean_angular_features = [
    "MeanYawAngularSpeed_Head_Start", "MeanYawAngularSpeed_Head_Mid", "MeanYawAngularSpeed_Head_End",
    "MeanPitchAngularSpeed_Head_Start", "MeanPitchAngularSpeed_Head_Mid", "MeanPitchAngularSpeed_Head_End",
    "MeanRollAngularSpeed_Head_Start", "MeanRollAngularSpeed_Head_Mid", "MeanRollAngularSpeed_Head_End",
    "Avg_MeanYawAngularSpeed_Hand_Start", "Avg_MeanYawAngularSpeed_Hand_Mid", "Avg_MeanYawAngularSpeed_Hand_End",
    "Avg_MeanPitchAngularSpeed_Hand_Start", "Avg_MeanPitchAngularSpeed_Hand_Mid", "Avg_MeanPitchAngularSpeed_Hand_End",
    "Avg_MeanRollAngularSpeed_Hand_Start", "Avg_MeanRollAngularSpeed_Hand_Mid", "Avg_MeanRollAngularSpeed_Hand_End"
]
std_angular_features = [
    "StdYawAngularSpeed_Head_Start", "StdYawAngularSpeed_Head_Mid", "StdYawAngularSpeed_Head_End",
    "StdPitchAngularSpeed_Head_Start", "StdPitchAngularSpeed_Head_Mid", "StdPitchAngularSpeed_Head_End",
    "StdRollAngularSpeed_Head_Start", "StdRollAngularSpeed_Head_Mid", "StdRollAngularSpeed_Head_End",                    
    "Avg_StdYawAngularSpeed_Hand_Start", "Avg_StdYawAngularSpeed_Hand_Mid", "Avg_StdYawAngularSpeed_Hand_End",
    "Avg_StdPitchAngularSpeed_Hand_Start", "Avg_StdPitchAngularSpeed_Hand_Mid", "Avg_StdPitchAngularSpeed_Hand_End",
    "Avg_StdRollAngularSpeed_Hand_Start", "Avg_StdRollAngularSpeed_Hand_Mid", "Avg_StdRollAngularSpeed_Hand_End"
]"""

"""subject_features = get_features_set('Doctor', '157015')
agent_features = get_features_set('Agent', '157015')
all_features = get_features_set('Doctor+Agent', '157015')
subject_features_nophase = get_features_set('Doctor', 'No Phase')
agent_features_nophase = get_features_set('Agent', 'No Phase')
all_features_nophase = get_features_set('Doctor+Agent', 'No Phase')
subject_features_verbal = get_features_set('Doctor', 'Verbal')
agent_features_verbal = get_features_set('Agent', 'Verbal')
all_features_verbal = get_features_set('Doctor+Agent', 'Verbal')
subject_features_nonverbal = get_features_set('Doctor', 'Non-Verbal')
agent_features_nonverbal = get_features_set('Agent', 'Non-Verbal')
all_features_nonverbal = get_features_set('Doctor+Agent', 'Non-Verbal')
subject_features_multimodal = get_features_set('Doctor', 'Multimodal')
agent_features_multimodal = get_features_set('Agent', 'Multimodal')
all_features_multimodal = get_features_set('Doctor+Agent', 'Multimodal')"""

In [ ]:


#out_path = os.path.join(config.OUT_PATH, 'gridsearchcv')
#if not os.path.exists(out_path): os.makedirs(out_path)
    
#OUT_PATH=os.path.join(config.OUT_PATH, 'cleaned-20190422')

In [ ]:
            """from sklearn.metrics import roc_curve, auc

            y_predict_proba = clf.predict_proba(X_test)

            # Compute ROC curve and ROC AUC for each class
            n_classes = 3
            fpr = dict()
            tpr = dict()
            roc_auc = dict()
            all_y_test_i = np.array([])
            all_y_predict_proba = np.array([])
            for i in range(n_classes):
                y_test_i = map(lambda x: 1 if x == i else 0, y_test)
                all_y_test_i = np.concatenate([all_y_test_i, y_test_i])
                all_y_predict_proba = np.concatenate([all_y_predict_proba, y_predict_proba[:, i]])
                fpr[i], tpr[i], _ = roc_curve(y_test_i, y_predict_proba[:, i])
                roc_auc[i] = auc(fpr[i], tpr[i])

            # Compute micro-average ROC curve and ROC area
            fpr["average"], tpr["average"], _ = roc_curve(all_y_test_i, all_y_predict_proba)
            roc_auc["average"] = auc(fpr["average"], tpr["average"])


            # Plot average ROC Curve
            plt.figure()
            plt.plot(fpr["average"], tpr["average"],
                     label='Average ROC curve (area = {0:0.2f})'
                           ''.format(roc_auc["average"]),
                     color='deeppink', linestyle=':', linewidth=4)

            # Plot each individual ROC curve
            for i in range(n_classes):
                plt.plot(fpr[i], tpr[i], lw=2,
                         label='ROC curve of class {0} (area = {1:0.2f})'
                         ''.format(i, roc_auc[i]))

            plt.plot([0, 1], [0, 1], 'k--', lw=2)
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('ROC curve %s / %s / test set ? %s / oversampling %s' % (clf.__class__.__name__, prediction_task, 
                                                                               test_set, upsample))
            plt.legend(loc="lower right")
            plt.show()"""

In [ ]:
        """
        # features importances
        if dropcol_fis is not None:
            final_fi = dropcol_fis.T
            final_fi['means'] = final_fi.mean(axis=1)
            final_fi['std'] = final_fi.std(axis=1)
            final_fi['sem'] = final_fi.sem(axis=1)
            final_fi['confidence'] = final_fi.apply(lambda row : 2*(np.mean(row) - st.t.interval(0.95, len(row)-1, loc=np.mean(row), scale=st.sem(row))[0]) , axis = 1)
            plot_importance('Columns Drop - ' + test_set_name, 
                            final_fi['means'],
                            final_fi['confidence'],
                            feats, 
                            sort=False)
            dumpPath = os.path.join(dh.root_path, 'results', 'features_importances_dropcol_%s.xlsx' % test_set_name)
            final_fi.to_excel(dumpPath, index=True)


        if fis is not None:
            final_fi_ = fis.T
            final_fi_['means'] = final_fi_.mean(axis=1)
            final_fi_['std'] = final_fi_.std(axis=1)
            final_fi_['sem'] = final_fi_.sem(axis=1)
            final_fi_['confidence'] = final_fi_.apply(lambda row : 2*(np.mean(row) - st.t.interval(0.95, len(row)-1, loc=np.mean(row), scale=st.sem(row))[0]) , axis = 1)
            plot_importance('Gini - ' + test_set_name, 
                            final_fi_['means'],
                            final_fi_['confidence'],
                            feats, 
                            sort=False)
            dumpPath = os.path.join(dh.root_path, 'results', 'features_importances_%s.xlsx' % test_set_name)
            final_fi.to_excel(dumpPath, index=True)"""

In [ ]:
# from run_gridsearch (if test_set: ...)

        else: # no test set - useless ?
            if verbose > 1:
                print('    Running %d 10-folds cv iterations on best classifier %s ...' % (repeat, best_params))

            global all_feature_importances
            global all_dropcol_feature_importances
            
            if grid is not None:
                best_clf = sklearn.base.clone(grid.best_estimator_) 
            else:
                best_clf = clf
            
            for i in np.arange(repeat):
                #forest = RandomForestClassifier(oob_score=True)
                # set best params to classifier

                scores = run_cross_val_score(best_clf, 
                                  X, y,  
                                  n_jobs=n_jobs,
                                  verbose=verbose)
                if verbose > 2:
                    print("iter # %d" % i)
                #    print(scores)
                scores_df = pd.DataFrame(scores)
                #print(scores_df)
                #print(scores_df.columns)
                all_scores.append(scores_df)



            # compute feature importance for random forest : we don't need validation data as it relies on oob score
            if type(clf) == RandomForestClassifier:
                all_feature_importances = pd.DataFrame(all_feature_importances, columns=features)        
                if fi:
                    X_ = pd.DataFrame(X, columns=features)
                    y_ = pd.DataFrame(y)        
                    for i in np.arange(repeat_dc):
                        # alternative computation for feature importance
                        all_dropcol_feature_importances.append(dropcol_importances(best_clf, X_, y_, RANDOM_STATE+i))
            else:
                all_feature_importances = None
                all_dropcol_feature_importances = None
                '''print(X, y)
                skf = StratifiedKFold(n_splits=repeat, random_state=RANDOM_STATE, shuffle=True)
                for i in np.arange(repeat_dc):
                    for train_index, test_index in skf.split(X, y):
                        X_train, X_test = X[train_index], X[test_index]
                        y_train, y_test = y[train_index], y[test_index]
                        X_train = pd.DataFrame(X_train, columns=features)
                        X_test = pd.DataFrame(X_test, columns=features)
                        y_train = pd.DataFrame(y_train)
                        y_test = pd.DataFrame(y_test)
                        # alternative computation for feature importance
                        dcs = dropcol_importances_sklearn(clf, X_train, y_train, X_test, y_test, RANDOM_STATE+i)
                        print("X_train %s X_test %s y_train %s y_test %s" % (len(X_train), len(X_test), len(y_train), len(y_test)))
                        print(dcs)
                        all_dropcol_feature_importances.append(dcs)'''

            #all_scores = pd.concat(all_scores)
            if all_dropcol_feature_importances is not None:
                all_dropcol_feature_importances = pd.DataFrame(all_dropcol_feature_importances, columns=features)

In [ ]:
# prepare results
def update_results_from_gridsearch(rdf, cv_results_, p1, p2, p3, p4, p5):
    df = pd.DataFrame(cv_results_)
    best_precision = df.mean_test_precision_macro.max()
    df_best = df.loc[df['mean_test_precision_macro'] == best_precision]
    if p5 == 'score':
        cols = [col for col in df_best.columns if ('mean_test_' in col)]
    elif p5 == 'err':
        cols = [col for col in df_best.columns if ('mean_std_' in col)]
    print('update_results_from_gridsearch: cols %s' % cols)
    update_results(rdf, df_best[cols], p1, p2, p3, p4, p5)
    return rdf

def update_results(rdf, df, p1, p2, p3, p4, p5):
    for i, col in enumerate(df.columns):
        rdf.loc[(p1, p2, p3, p4, p5), col] = df.iloc[0].loc[col]
    return rdf


def update_results_single(rdf, df, p1, p2, p3, p4, p5):
    for i, col in enumerate(df.columns):
        rdf.loc[(p1, p2, p3, p4, p5), col] = df.loc[col]
    return rdf

In [ ]:
# from run_cross_val_score()

#if not isinstance(clf, JNCC2Wrapper) and not (isinstance(clf, Pipeline) and isinstance(clf.named_steps['clf'], JNCC2Wrapper)):
        
    """else:
        labels = sorted(np.unique(y).astype(int))        
        cross_val_scores = {}
        cross_val_scores['val_f1_macro_weighted'] = []
        for idx, label in enumerate(labels):
            cross_val_scores['val_f1_macro_weighted_class_%d' % label] = []
        cross_val_scores['val_precision_macro_weighted'] = []
        for idx, label in enumerate(labels):
            cross_val_scores['val_precision_macro_weighted_class_%d' % label] = []   
        cross_val_scores['val_recall_macro_weighted'] = []
        for idx, label in enumerate(labels):
            cross_val_scores['val_recall_macro_weighted_class_%d' % label] = []     
        cross_val_score['val_f1_micro'] = []
        cross_val_scores['val_mrr'] = []
        
            
        jncc_cv, y_trues_cv = clf.cv('train.arff', idx)
        
        cross_val_scores['score_time'] = [0]*len(jncc_cv)
        
        for i in np.arange(len(jncc_cv)):
            y_true = y_trues_cv[i]
            y_pred = jncc_cv[i][:, 0]
            cross_val_scores['val_f1_macro_weighted'].append(f1_score(y_true, y_pred, average='weighted'))
            for idx, label in enumerate(labels):
                cross_val_scores['val_f1_macro_weighted_class_%d' % label].append(f1_score(y_true, y_pred, labels=[label], average='weighted'))
            cross_val_scores['val_precision_macro_weighted'].append(precision_score(y_true, y_pred, average='weighted'))
            for idx, label in enumerate(labels):
                cross_val_scores['val_precision_macro_weighted_class_%d' % label].append(precision_score(y_true, y_pred, labels=[label], average='weighted'))    
            cross_val_scores['val_recall_macro_weighted'].append(recall_score(y_true, y_pred, average='weighted'))
            for idx, label in enumerate(labels):
                cross_val_scores['val_recall_macro_weighted_class_%d' % label].append(recall_score(y_true, y_pred, labels=[label], average='weighted'))    
            #print('y_true %s y_pred %s' % (y_true.shape, jncc_cv[i].shape))
            cross_val_scores['val_f1_micro'].append(f1_score(y_true, y_pred, average='micro'))
            cross_val_scores['val_mrr'].append(feu.compute_mrr(y_true, jncc_cv[i], labels))"""

In [ ]:
"""
def run_gridsearch(clf, param_grid, data, features, prediction_task, repeat=10, repeat_dc=300, test_set=0.1, n_splits=1, 
                   upsample=None, fi=False, bins=None, strategy=None, force_gs=True, n_jobs=-1, verbose=0):
    if verbose > 1:
        print('run_gridsearch(clf=%s, param_grid=%s, bins=%d, strategy=%s)' % (clf.__class__.__name__, str(param_grid), 
                                                                               bins, strategy))
    X_test = None
    y_test = None
    create_test_set = test_set is not None and test_set > 0
    best_params = None
    grid = None
    is_jncc = isinstance(get_clf_from_estimator(clf), JNCC2Wrapper)
    
    all_scores = []
    all_test_scores = []    
    all_supports = []
    all_test_supports = []
    fi_method = []
    fi_fs = pd.DataFrame(np.zeros((1, len(features_df.columns))), columns=features_df.columns)
    
    splits = prepare_train_data(samples=data, features=features, prediction_task=prediction_task, test_set=test_set, 
                                n_splits=n_splits, upsample=upsample, bins=bins, strategy=strategy, verbose=verbose)

    for idx, (X, y, X_test, y_test) in enumerate(splits):
        if verbose > 1:
            print("*** Train/Test Split #%d" % idx)
        y = y.astype(int)
        support = np.unique(y, return_counts=True)
        support_test = None
        if y_test is not None:
            y_test = y_test.astype(int)
            support_test = np.unique(y_test, return_counts=True)
        all_supports.append(str(support))
        all_test_supports.append(str(support_test))
        
        # first evaluate best params through grid search for this particular dataset/featureset
        # !!! important, see http://www.jmlr.org/papers/volume11/cawley10a/cawley10a.pdf
        # use 'force_gs' so hyper-parameters search is performed independently on each train split before evaluation on test, which would be unbiased protocol
        if best_params is None or force_gs :
            if len(ParameterGrid(param_grid)) > 1:
                grid = gridsearch2(clf, 
                                  X, y, 
                                  prediction_task, 
                                  param_grid=param_grid, 
                                  features=features,
                                  n_jobs=n_jobs if not is_jncc else 1, # avoid multithreading with JNCCWrapper
                                  verbose=verbose)
                best_params = grid.best_params_
                single_best_score = grid.best_score_
                if verbose > 1:
                    print('  Gridsearch evaluated best score %s' % (single_best_score))
                    print('  Gridsearch best params : %s' % str(best_params))

            else:
                if verbose > 1:
                    print('run_gridsearch: only 1 set of parameters, gridsearch not required')
                best_params = ParameterGrid(param_grid)[0]

        if create_test_set:

            # compute cv scores (redundant with gridsearch ?)
            
            #if not is_jncc:

            if grid is not None:
                best_clf = sklearn.base.clone(grid.best_estimator_)
            else:
                best_clf = clf
                if verbose > 2:
                    print('CLF possible params %s' % str(clf.get_params().keys()))
                best_clf.set_params(**best_params)
            ""if isinstance(best_clf, JNCC2Wrapper):
                # pass the DataHandler
                best_clf = JNCC2Wrapper(dh)
            elif isinstance(best_clf, Pipeline):
                if 'clf' in best_clf.named_steps and isinstance(best_clf.named_steps['clf'], JNCC2Wrapper):
                    # pass the DataHandler to the pipelined JNCC2
                    best_clf.set_params(clf__dataHandler=dh)   
                print('DEBUG pipeline named steps %s' % str(best_clf.named_steps))  ""                  
            val_scores = run_cross_val_score(best_clf, X, y, n_jobs=n_jobs, verbose=verbose)
                            
            #else:
                
            #    val_scores = run_cross_val_score(clf, X, y, idx=idx, verbose=verbose)
                
                
    
            #jncc_result, y_true_predict = jncc.predict('train.arff', 'test.arff', 0)
                
            val_scores = pd.DataFrame(val_scores)
            all_scores.append(val_scores)

            # compute test scores
            
            labels = sorted(support[0])
            if verbose > 2:
                print("LABELS " + str(labels))            

            #if not is_jncc:
            
            if grid is not None:
                best_clf = sklearn.base.clone(grid.best_estimator_)
            else:
                best_clf = clf

            best_clf.fit(X, y)
            y_pred = best_clf.predict(X_test)
            
            if is_jncc:
                # jncc returns a multi-label proposition
                jncc_result = y_pred
                if y_pred.values.ndim == 1:
                    y_pred = jncc_result
                else:
                    y_pred = jncc_result.iloc(axis=1)[0]

            # retrieve feature selection information, if any was performed
            if type(best_clf) == Pipeline and 'fs' in best_clf.named_steps:
                selected_features = get_feature_names_from_pipeline(best_clf, features)['fs']
                if selected_features is not None:
                    if verbose > 1:
                        print('  Selected features %s' % str(selected_features))
                    for ifeat in selected_features:
                        fi_fs[ifeat] += 1                          
                
            #else:
            #    # for JNCC (multilabel result), usual scores are computed thanks to first prediction, and MRR is computed based on ranked predictions
            #    jncc_result, y_true = clf.predict('train.arff', 'test.arff', idx)
            #    y_pred = jncc_result.iloc(axis=1)[0]
            
            
            test_scores_row = {
                'test_balanced_acc': balanced_accuracy_score(y_test, y_pred),
                #'test_balanced_acc_adjusted': balanced_accuracy_score(y_test, y_pred, adjusted=True),
                'test_f1_macro_weighted': f1_score(y_test, y_pred, average='weighted') }
            for idx, label in enumerate(labels):
                test_scores_row['test_f1_macro_class_%d' % label] = f1_score(y_test, y_pred, labels=[label], average='weighted')
            test_scores_row['test_precision_macro_weighted'] =  precision_score(y_test, y_pred, average='weighted')
            for idx, label in enumerate(labels):
                test_scores_row['test_precision_macro_class_%d' % label] = precision_score(y_test, y_pred, labels=[label], average='weighted')            
            test_scores_row['test_recall_macro_weighted'] =  recall_score(y_test, y_pred, average='weighted')
            for idx, label in enumerate(labels):
                test_scores_row['test_recall_macro_class_%d' % label] = recall_score(y_test, y_pred, labels=[label], average='weighted')     
            #test_scores_row['test_f1_micro'] = f1_score(y_test, y_pred, average='micro')
            if is_jncc:
                test_scores_row['test_mrr'] = feu.compute_mrr(y_test, jncc_result, labels)
                test_scores_row['test_determinacy'] = feu.compute_determinacy(jncc_result)
                # TODO: add scores performed by JNCC2 ? (single accuracy etc)
                
            # Compute dummy classifier scores using different strategies, as a baseline
            for dummy_strat in ['uniform', 'stratified', 'most_frequent']:
                dummy = DummyClassifier(strategy=dummy_strat, random_state=RANDOM_STATE)
                dummy.fit(X, y)
                y_pred = dummy.predict(X_test)
                test_scores_row ['rnd_%s_balanced_acc' % dummy_strat] = balanced_accuracy_score(y_test, y_pred)
                test_scores_row['rnd_%s_f1_macro_weighted' % dummy_strat] = f1_score(y_test, y_pred, average='weighted')
                for idx, label in enumerate(labels):
                    test_scores_row['rnd_%s_f1_macro_class_%d' % (dummy_strat, label)] = f1_score(y_test, y_pred, labels=[label], average='weighted')
                test_scores_row['rnd_%s_precision_macro_weighted' % dummy_strat] =  precision_score(y_test, y_pred, average='weighted')
                for idx, label in enumerate(labels):
                    test_scores_row['rnd_%s_precision_macro_class_%d' % (dummy_strat, label)] = precision_score(y_test, y_pred, labels=[label], average='weighted')            
                test_scores_row['rnd_%s_recall_macro_weighted' % dummy_strat] =  recall_score(y_test, y_pred, average='weighted')
                for idx, label in enumerate(labels):
                    test_scores_row['rnd_%s_recall_macro_class_%d' % (dummy_strat, label)] = recall_score(y_test, y_pred, labels=[label], average='weighted')                  
                test_scores_row['rnd_%s_f1_micro' % dummy_strat] = f1_score(y_test, y_pred, average='micro')
                    
            test_scores_df = pd.DataFrame([test_scores_row], index=['score'])
            all_test_scores.append(test_scores_df)


        if not create_test_set:
            if verbose > 1:
                print('    Running %d 10-folds cv iterations on best classifier %s ...' % (repeat, best_params))

            global all_feature_importances
            global all_dropcol_feature_importances
            
            if grid is not None:
                best_clf = sklearn.base.clone(grid.best_estimator_) 
            else:
                best_clf = clf
            
            for i in np.arange(repeat):
                #forest = RandomForestClassifier(oob_score=True)
                # set best params to classifier

                scores = run_cross_val_score(best_clf, 
                                  X, y,  
                                  n_jobs=n_jobs,
                                  verbose=verbose)
                if verbose > 2:
                    print("iter # %d" % i)
                #    print(scores)
                scores_df = pd.DataFrame(scores)
                #print(scores_df)
                #print(scores_df.columns)
                all_scores.append(scores_df)



            # compute feature importance for random forest : we don't need validation data as it relies on oob score
            if type(clf) == RandomForestClassifier:
                all_feature_importances = pd.DataFrame(all_feature_importances, columns=features)        
                if fi:
                    X_ = pd.DataFrame(X, columns=features)
                    y_ = pd.DataFrame(y)        
                    for i in np.arange(repeat_dc):
                        # alternative computation for feature importance
                        all_dropcol_feature_importances.append(dropcol_importances(best_clf, X_, y_, RANDOM_STATE+i))
            else:
                all_feature_importances = None
                all_dropcol_feature_importances = None
                '''print(X, y)
                skf = StratifiedKFold(n_splits=repeat, random_state=RANDOM_STATE, shuffle=True)
                for i in np.arange(repeat_dc):
                    for train_index, test_index in skf.split(X, y):
                        X_train, X_test = X[train_index], X[test_index]
                        y_train, y_test = y[train_index], y[test_index]
                        X_train = pd.DataFrame(X_train, columns=features)
                        X_test = pd.DataFrame(X_test, columns=features)
                        y_train = pd.DataFrame(y_train)
                        y_test = pd.DataFrame(y_test)
                        # alternative computation for feature importance
                        dcs = dropcol_importances_sklearn(clf, X_train, y_train, X_test, y_test, RANDOM_STATE+i)
                        print("X_train %s X_test %s y_train %s y_test %s" % (len(X_train), len(X_test), len(y_train), len(y_test)))
                        print(dcs)
                        all_dropcol_feature_importances.append(dcs)'''

            #all_scores = pd.concat(all_scores)
            if all_dropcol_feature_importances is not None:
                all_dropcol_feature_importances = pd.DataFrame(all_dropcol_feature_importances, columns=features)

    if len(all_scores) == 1:
        all_scores = all_scores[0]
    elif len(all_scores) > 0:
        all_scores = pd.concat(all_scores)
    else:
        all_scores = pd.DataFrame()
    if len(all_test_scores) == 1:
        all_test_scores = all_test_scores[0]
    elif len(all_test_scores) > 0:
        all_test_scores = pd.concat(all_test_scores)
    else:
        all_test_scores = pd.DataFrame()
        
    result = {
        'all_scores': all_scores,
        'all_test_scores': all_test_scores,
        'dropcol_fi': all_dropcol_feature_importances,
        'fi': all_feature_importances,
        'fi_fs': fi_fs,
        'support': support,
        'support_test': support_test
    }
        
    return grid, result
"""

In [ ]:
            y_pred = y_pred_rnd[:, best]
            test_scores_row ['rnd_%s_balanced_acc' % dummy_strat] = balanced_accuracy_score(y_test, y_pred)
            test_scores_row['rnd_%s_f1_macro_weighted' % dummy_strat] = f1_score(y_test, y_pred, average='weighted')
            for idx, label in enumerate(labels):
                test_scores_row['rnd_%s_f1_macro_class_%d' % (dummy_strat, label)] = f1_score(y_test, y_pred, labels=[label], average='weighted')
            test_scores_row['rnd_%s_precision_macro_weighted' % dummy_strat] =  precision_score(y_test, y_pred, average='weighted')
            for idx, label in enumerate(labels):
                test_scores_row['rnd_%s_precision_macro_class_%d' % (dummy_strat, label)] = precision_score(y_test, y_pred, labels=[label], average='weighted')            
            test_scores_row['rnd_%s_recall_macro_weighted' % dummy_strat] =  recall_score(y_test, y_pred, average='weighted')
            for idx, label in enumerate(labels):
                test_scores_row['rnd_%s_recall_macro_class_%d' % (dummy_strat, label)] = recall_score(y_test, y_pred, labels=[label], average='weighted')                  
            test_scores_row['rnd_%s_f1_micro' % dummy_strat] = f1_score(y_test, y_pred, average='micro')